In [144]:
import pandas as pd

In [145]:
df = pd.read_csv('data/main/charger.csv')
df["timestamp"] = pd.to_datetime(df["timestamp"])
df["timestamp"] = df["timestamp"].dt.tz_convert(
    "Europe/Oslo"
)
gdf = df.groupby(['site_key', 'timestamp']).sum('value').reset_index()
gdf['site_key'].unique().size

584

In [146]:
gdf['day'] = gdf['timestamp'].dt.day
gdf['month'] = gdf['timestamp'].dt.month
gdf['year'] = gdf['timestamp'].dt.year
# gdf[gdf['month'] == 12].sort_values('value', ascending=False).head(5)[["timestamp", "value"]]

In [147]:
gdf1 = gdf.groupby(['site_key',  'year', 'month','day',])['value'].max().reset_index()
gdf2 = gdf1.groupby(['site_key', 'year','month', ])['value'].nlargest(3).reset_index()
print(gdf2)
gdf3 = gdf2.groupby(['site_key', 'year','month'])['value'].mean().reset_index()
print(gdf3)

gdf3.to_csv('output.csv', index=False)

       site_key  year  month  level_3      value
0     24U4-G222  2024     12        1   7.571368
1     24U4-G222  2024     12       14   7.566715
2     24U4-G222  2024     12        8   7.553243
3     24U4-G222  2025      1       35   7.539638
4     24U4-G222  2025      1       31   7.524898
...         ...   ...    ...      ...        ...
3466  ZZUF-H222  2024     12    27219  16.849098
3467  ZZUF-H222  2024     12    27220  16.560462
3468  ZZUF-H222  2025      1    27231  12.060527
3469  ZZUF-H222  2025      1    27232  11.381672
3470  ZZUF-H222  2025      1    27245  11.002890

[3471 rows x 5 columns]
       site_key  year  month        value
0     24U4-G222  2024     12     7.563775
1     24U4-G222  2025      1     7.526582
2     28GD-L522  2024     12    52.947657
3     28GD-L522  2025      1    88.455922
4     28T4-7722  2024     12    91.144472
...         ...   ...    ...          ...
1158  ZXKY-B622  2025      1     2.626631
1159  ZYY4-R522  2024     12    22.328309
1160  ZYY